In [1]:
import cv2
import time
import csv
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


In [ ]:
model = YOLO("my_model.pt")

tracker = DeepSort(max_age=30)


cap = cv2.VideoCapture('videos/clean.mp4')  
fps = cap.get(cv2.CAP_PROP_FPS)
frame_index = 0

# Data store for logging
tracking_data = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_index += 1
    timestamp = frame_index / fps

    resized_frame = cv2.resize(frame, (640, 640))
    results = model(resized_frame)[0]

    detections = []
    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls_id))

   
    tracks = tracker.update_tracks(detections, frame=resized_frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        center_x = (l + r) // 2

        # Log first appearance
        if track_id not in tracking_data:
            tracking_data[track_id] = {
                "entry_time": timestamp,
                "positions": [center_x]
            }
        else:
            tracking_data[track_id]["positions"].append(center_x)

        # Draw rectangle and label
        cv2.rectangle(resized_frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(resized_frame, f'Bird ID {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Bird Tracking", resized_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


with open('bird_tracking_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['Bird ID', 'Entry Time (s)', 'Movement']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for track_id, data in tracking_data.items():
        entry_time = round(data['entry_time'], 2)
        positions = data['positions']
        movement = "Stayed"

        if positions[-1] - positions[0] > 30:
            movement = "Left to Right"
        elif positions[0] - positions[-1] > 30:
            movement = "Right to Left"

        writer.writerow({
            'Bird ID': track_id,
            'Entry Time (s)': entry_time,
            'Movement': movement
        })

print("Tracking log saved to bird_tracking_log.csv")


In [ ]:


model = YOLO("my_model.pt")
tracker = DeepSort(max_age=30)

cap = cv2.VideoCapture("videos/clean.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_index = 0
SKIP_FRAMES = 3

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps // SKIP_FRAMES, (640, 640))

tracking_data = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_index += 1
    if frame_index % SKIP_FRAMES != 0:
        continue

    timestamp = frame_index / fps
    resized_frame = cv2.resize(frame, (640, 640))
    results = model(resized_frame)[0]

    detections = []
    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls_id))

    tracks = tracker.update_tracks(detections, frame=resized_frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        center_x = (l + r) // 2

        if track_id not in tracking_data:
            tracking_data[track_id] = {
                "entry_time": timestamp,
                "positions": [center_x]
            }
        else:
            tracking_data[track_id]["positions"].append(center_x)

        cv2.rectangle(resized_frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(resized_frame, f'Bird ID {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(resized_frame)

cap.release()
out.release()
cv2.destroyAllWindows()

with open('bird_tracking_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['Bird ID', 'Entry Time (s)', 'Movement']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for track_id, data in tracking_data.items():
        entry_time = round(data['entry_time'], 2)
        positions = data['positions']
        movement = "Stayed"

        if positions[-1] - positions[0] > 30:
            movement = "Left to Right"
        elif positions[0] - positions[-1] > 30:
            movement = "Right to Left"

        writer.writerow({
            'Bird ID': track_id,
            'Entry Time (s)': entry_time,
            'Movement': movement
        })

print("✅ Output video saved as 'output_video.mp4'")
print("📄 Tracking log saved as 'bird_tracking_log.csv'")


In [7]:
import pandas as pd

df = pd.read_csv('bird_tracking_log.csv')
df

,Bird ID,Entry Time (s),Movement
0,1,0.30,Left to Right
1,2,0.30,Left to Right
2,3,0.30,Stayed
3,4,0.30,Left to Right
4,5,0.30,Right to Left
5,6,0.30,Left to Right
6,7,0.30,Right to Left
7,11,0.80,Left to Right
8,13,1.20,Left to Right
9,28,2.60,Left to Right
